In [30]:
import numpy as np
import sys
sys.path.insert(1, './fashion-mnist/utils')
import mnist_reader

In [31]:
x_train, y_train = mnist_reader.load_mnist('./fashion-mnist/data/fashion', kind='train')
x_test, y_test = mnist_reader.load_mnist('./fashion-mnist/data/fashion', kind='t10k')
print('x shape:', x_train.shape)
print('y shape:', y_train.shape)

x shape: (60000, 784)
y shape: (60000,)


In [32]:
# Normalizar datos, 255 es el valor maximo de RGB
x_train = x_train / 255

In [33]:
# Transformar vector y, para reflejar nuestras 10 neuronas de salida
y_trains = []
for i in y_train:
    y = []
    for j in range(0, 10):
        if i == j:
            y.append(1.0)
        else:
            y.append(0.0)
    y_trains.append(y)

y_trains = np.array(y_trains)
y_trains.shape

(60000, 10)

In [34]:
class NeuralNet:
    def __init__(self, x, y):
        np.random.seed(69)
        self.x = x
        self.y = y
        self.learning_rate = 0.01
        n = 1 / np.sqrt(x.shape[1])
        self.w_ih = np.random.uniform(low=-n, high=n, size=(x.shape[1] // 3, x.shape[1]))
        self.w_ho = np.random.uniform(low=-n, high=n, size=(y.shape[1], x.shape[1] // 3))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
        
    def feed_forward(self, x):
        # Capa input a hidden
        #z_ih = np.matmul(self.w_ih, self.x.T)
        z_ih = np.matmul(self.w_ih, x.T)
        self.a_ih = self.sigmoid(z_ih)
        
        # Capa hidden a output
        z_ho = np.matmul(self.w_ho, self.a_ih)
        self.a_ho = self.sigmoid(z_ho)
        
    def back_propagate(self, x, y):
        #error_o = self.y.T - self.a_ho
        error_o = y.T - self.a_ho
        error_od = error_o * (self.a_ho * (1.0 - self.a_ho))

        ho_error = np.matmul(self.w_ho.T, error_od)
        ho_d = ho_error * (self.a_ih * (1.0 - self.a_ih))

        #self.w_ih += self.learning_rate * np.matmul(ho_d, self.x)
        self.w_ih += self.learning_rate * np.matmul(ho_d, x)
        self.w_ho += self.learning_rate * np.matmul(error_od, self.a_ih.T)
        
    def train(self, x, y):
        self.feed_forward(x)
        self.back_propagate(x, y)
            
    def predict(self, x):
        z_ih = np.matmul(self.w_ih, x.T)
        self.a_ih = self.sigmoid(z_ih)
        z_ho = np.matmul(self.w_ho, self.a_ih)
        return self.sigmoid(z_ho)

In [38]:
def score(x_test, y_test):
    correct = 0
    wrong = 0
    for x in range(len(x_test)):
        y_pred = net.predict(x_test[x])
        highest = 0.0
        highest_index = 0
        for y in range(len(y_pred)):
            if y_pred[y] > highest:
                highest = y_pred[y]
                highest_index = y
        if highest_index == y_test[x]:
            correct += 1
        else:
            wrong += 1

    return (correct / len(x_test))

In [39]:
net = NeuralNet(x_train, y_trains)

epochs = 3000
size = 100
highest_score = 0.0
for e in range(epochs):
    low = 0
    limit = 60000 - size
    while low < limit:
        x = x_train[low : low + size]
        y = y_trains[low : low + size]
        net.train(x, y)
        low += size
    curr_score = score(x_test[0:100], y_test[0:100])
    if curr_score > highest_score:
        highest_score = curr_score
        print(f'Epoch {e} finished, new high score is {highest_score}')

/Users/sebdev6992/Documents/uvg/2020/s1/ai/lab2/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == '':


Epoch 0 finished, new high score is 0.66
Epoch 1 finished, new high score is 0.69
Epoch 2 finished, new high score is 0.75
Epoch 13 finished, new high score is 0.76
Epoch 16 finished, new high score is 0.78
Epoch 17 finished, new high score is 0.79
Epoch 18 finished, new high score is 0.8
Epoch 23 finished, new high score is 0.82
Epoch 30 finished, new high score is 0.83
Epoch 31 finished, new high score is 0.85
Epoch 33 finished, new high score is 0.86
Epoch 36 finished, new high score is 0.87
Epoch 38 finished, new high score is 0.88
Epoch 57 finished, new high score is 0.89
